In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install unbabel-comet
!pip install evaluate
!pip install sympy==1.12
!pip install sacrebleu
!pip install bert_score

  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.12 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 66.8 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successfully uninstalled sympy-1.12


In [18]:
prediction_file_name = "gtranslate.txt"
MODEL_NAME = "GTranslate"

In [19]:
prediction_folders = []

prediction_folders.append(f"/content/drive/MyDrive/disertatie/translate/{MODEL_NAME}/arabia_saudita_guvern")
prediction_folders.append(f"/content/drive/MyDrive/disertatie/translate/{MODEL_NAME}/conventia_geneva")
prediction_folders.append(f"/content/drive/MyDrive/disertatie/translate/{MODEL_NAME}/drepturile_copilului")
prediction_folders.append(f"/content/drive/MyDrive/disertatie/translate/{MODEL_NAME}/drepturile_economice_culturale")
prediction_folders.append(f"/content/drive/MyDrive/disertatie/translate/{MODEL_NAME}/drepturile_omului")
prediction_folders.append(f"/content/drive/MyDrive/disertatie/translate/{MODEL_NAME}/twitter/ksaembassyrou")
prediction_folders.append(f"/content/drive/MyDrive/disertatie/translate/{MODEL_NAME}/statutul_refugiatilor")
prediction_folders.append(f"/content/drive/MyDrive/disertatie/translate/{MODEL_NAME}/flores+")
prediction_folders.append(f"/content/drive/MyDrive/disertatie/translate/{MODEL_NAME}/from_training_dataset")

reference_folders = []
reference_file_name = "ro.txt"
reference_folders.append("/content/drive/MyDrive/disertatie/dataset/arabia_saudita_guvern")
reference_folders.append("/content/drive/MyDrive/disertatie/dataset/conventia_geneva")
reference_folders.append("/content/drive/MyDrive/disertatie/dataset/drepturile_copilului")
reference_folders.append("/content/drive/MyDrive/disertatie/dataset/drepturile_economice_culturale")
reference_folders.append("/content/drive/MyDrive/disertatie/dataset/drepturile_omului")
reference_folders.append("/content/drive/MyDrive/disertatie/dataset/twitter/ksaembassyrou")
reference_folders.append("/content/drive/MyDrive/disertatie/dataset/statutul_refugiatilor")
reference_folders.append("/content/drive/MyDrive/disertatie/dataset/flores+")
reference_folders.append("/content/drive/MyDrive/disertatie/dataset/from_training_dataset")

src_folders = []
src_file_name = "ar.txt"
src_folders.append("/content/drive/MyDrive/disertatie/dataset/arabia_saudita_guvern")
src_folders.append("/content/drive/MyDrive/disertatie/dataset/conventia_geneva")
src_folders.append("/content/drive/MyDrive/disertatie/dataset/drepturile_copilului")
src_folders.append("/content/drive/MyDrive/disertatie/dataset/drepturile_economice_culturale")
src_folders.append("/content/drive/MyDrive/disertatie/dataset/drepturile_omului")
src_folders.append("/content/drive/MyDrive/disertatie/dataset/twitter/ksaembassyrou")
src_folders.append("/content/drive/MyDrive/disertatie/dataset/statutul_refugiatilor")
src_folders.append("/content/drive/MyDrive/disertatie/dataset/flores+")
src_folders.append("/content/drive/MyDrive/disertatie/dataset/from_training_dataset")

In [20]:
results_dict = dict()

# CHRF++

In [21]:

import evaluate
import pandas as pd

def get_chrf_score(predictions_list, reference_list):
    chrf = evaluate.load("chrf")
    print(len(predictions_list), len(reference_list))
    results = chrf.compute(predictions=predictions_list, references=reference_list)
    return results

def read_and_get_results_CHRF(reference_folders, prediction_folders):
  results_dict["CHRF"] = dict()
  for i in range(len(reference_folders)):
    reference_file_path = reference_folders[i] + "/" + reference_file_name
    prediction_file_path = prediction_folders[i] + "/" + prediction_file_name

    name_of_dataset = reference_folders[i].split("/")[-1]

    lines_reference = None
    lines_prediction = None

    with open(reference_file_path, "r") as file:
        lines_reference = [line.strip() for line in file]

    with open(prediction_file_path, "r") as file:
        lines_prediction = [line.strip() for line in file if line.strip()]
        #lines_prediction = [
        #    line.strip() if line.strip() else " "  # Replace empty lines with a space
        #    for line in file
        #]

    score = get_chrf_score(lines_prediction, lines_reference)
    print(  name_of_dataset + ": " + str(score["score"]))
    results_dict["CHRF"][name_of_dataset] = score["score"]

In [23]:
read_and_get_results_CHRF(reference_folders, prediction_folders)

189 189
arabia_saudita_guvern: 62.90511402127397
330 330
conventia_geneva: 57.43893424896419
217 217
drepturile_copilului: 57.94840056393361
124 124
drepturile_economice_culturale: 59.95231514685165
79 79
drepturile_omului: 61.925131689194366
86 86
ksaembassyrou: 70.76927802578898
177 177
statutul_refugiatilor: 50.08552529508774
997 997
flores+: 60.09284757712173
2100 2100
from_training_dataset: 54.41370551775195


# BLEU

In [24]:
import evaluate
import pandas as pd

def get_bleu_score(predictions_list, reference_list):
    bleu = evaluate.load("bleu")
    print(len(predictions_list), len(reference_list))
    results = bleu.compute(predictions=predictions_list, references=reference_list)
    return results

def read_and_get_results_BLEU(reference_folders, prediction_folders):
  results_dict["BLEU"] = dict()
  for i in range(len(reference_folders)):
    reference_file_path = reference_folders[i] + "/" + reference_file_name
    prediction_file_path = prediction_folders[i] + "/" + prediction_file_name

    name_of_dataset = reference_folders[i].split("/")[-1]

    lines_reference = None
    lines_prediction = None

    with open(reference_file_path, "r") as file:
      lines_reference = [line.strip() for line in file if line.strip()]

    with open(prediction_file_path, "r") as file:
        lines_prediction = [line.strip() for line in file if line.strip()]
        #lines_prediction = [
        #    line.strip() if line.strip() else " "  # Replace empty lines with a space
        #    for line in file
        #]
#
    score = get_bleu_score(lines_prediction, lines_reference)
    print(  name_of_dataset + ": " + str(score['bleu'] * 100))
    results_dict["BLEU"][name_of_dataset] = score['bleu'] * 100

In [25]:
read_and_get_results_BLEU(reference_folders, prediction_folders)

189 189
arabia_saudita_guvern: 30.86491656321071
330 330
conventia_geneva: 21.94016492438294
217 217
drepturile_copilului: 22.772241277688295
124 124
drepturile_economice_culturale: 20.06376675238987
79 79
drepturile_omului: 28.110648543983867
86 86
ksaembassyrou: 39.17595543760482
177 177
statutul_refugiatilor: 14.356379916435932
997 997
flores+: 34.15940430058995
2100 2100
from_training_dataset: 27.711420585692338


# COMET

In [26]:
from comet import download_model, load_from_checkpoint
import pandas as pd
import numpy as np

model_path = download_model("Unbabel/wmt22-comet-da")

def read_and_get_results_COMET(reference_folders, prediction_folders, src_folders):
  results_dict["COMET"] = dict()
  for i in range(len(reference_folders)):
    reference_file_path = reference_folders[i] + "/" + reference_file_name
    prediction_file_path = prediction_folders[i] + "/" + prediction_file_name
    src_file_path = src_folders[i] + "/" + src_file_name

    name_of_dataset = reference_folders[i].split("/")[-1]

    lines_reference = None
    lines_prediction = None
    lines_src = None

    with open(reference_file_path, "r") as file:
        lines_reference = [line.strip() for line in file]

    with open(prediction_file_path, "r") as file:
        lines_prediction = [line.strip() for line in file if line.strip()]
        #lines_prediction = [
        #    line.strip() if line.strip() else " "  # Replace empty lines with a space
        #    for line in file
        #]
#
    with open(src_file_path, "r") as file:
        lines_src = [line.strip() for line in file]

    score = create_data_for_comet(lines_reference, lines_src, lines_prediction)
    print(  name_of_dataset + ": " + str(np.mean(score[0])), score)
    results_dict["COMET"][name_of_dataset] = np.mean(score[0])


def create_data_for_comet(reference_list, src_list, machine_translation_list):
    model = load_from_checkpoint(model_path)
    data = []

    for i in range(len(reference_list)):
        data_point = dict()
        data_point["src"] = src_list[i]
        data_point["ref"] = reference_list[i]
        data_point["mt"] = machine_translation_list[i]
        data.append(data_point)

    results = model.predict(data, batch_size=64, gpus=1)
    return results

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [27]:
read_and_get_results_COMET(reference_folders, prediction_folders, src_folders)

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_

arabia_saudita_guvern: 0.8445258583656694 Prediction([('scores', [0.5602481961250305, 0.9280099272727966, 0.8632186055183411, 0.8990617394447327, 0.9077911972999573, 0.9390187859535217, 0.5925443172454834, 0.4717271625995636, 0.8177122473716736, 0.9464697241783142, 0.5704477429389954, 0.7143866419792175, 0.7274044156074524, 0.8819515109062195, 0.882232129573822, 0.9578516483306885, 0.82851642370224, 0.7184513807296753, 0.7608135342597961, 0.9315829277038574, 0.9073795080184937, 0.910743772983551, 0.9166913628578186, 0.870016872882843, 0.5767183899879456, 0.8646731972694397, 0.8732740879058838, 0.9171761870384216, 0.7844060063362122, 0.7893754839897156, 0.8393166661262512, 0.6667551398277283, 0.9347702860832214, 0.8772892355918884, 0.8633626103401184, 0.8994721174240112, 0.8062184453010559, 0.9085800051689148, 0.9542263150215149, 0.8979322910308838, 0.9220802187919617, 0.9251537322998047, 0.8463228344917297, 0.9177791476249695, 0.8341392278671265, 0.9319440722465515, 0.9100045561790466,

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_

conventia_geneva: 0.8603398885690805 Prediction([('scores', [0.8723165392875671, 0.7826114296913147, 0.8163273930549622, 0.8769564628601074, 0.9394630789756775, 0.8024545311927795, 0.8978204727172852, 0.9303659796714783, 0.8537307381629944, 0.9935590028762817, 0.9178864359855652, 0.8941546082496643, 0.8301147818565369, 0.8667245507240295, 0.8734963536262512, 0.8418595194816589, 0.7902733683586121, 0.8762574195861816, 0.9310837388038635, 0.8755987286567688, 0.8574323654174805, 0.7437896728515625, 0.9292911291122437, 0.856960117816925, 0.8633822202682495, 0.8174930214881897, 0.8581823706626892, 0.8731465339660645, 0.8698787093162537, 0.9192411303520203, 0.9121771454811096, 0.7836177349090576, 0.8634991645812988, 0.748473584651947, 0.9244977831840515, 0.803084135055542, 0.8830798268318176, 0.9076239466667175, 0.9020949006080627, 0.855329692363739, 0.8864451050758362, 0.8608449101448059, 0.8212394714355469, 0.8545132279396057, 0.8326800465583801, 0.8430480360984802, 0.8329179883003235, 0.8

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_

...

statutul_refugiatilor: 0.8458502807859647 Prediction([('scores', [0.8694480061531067, 0.7609391212463379, 0.545308530330658, 0.6119442582130432, 0.40001192688941956, 0.6397095322608948, 0.6613251566886902, 0.5505930781364441, 0.37325337529182434, 0.750515878200531, 0.618444561958313, 0.5779996514320374, 0.5593782067298889, 0.7025884389877319, 0.5721742510795593, 0.7798661589622498, 0.8783140778541565, 0.8528776168823242, 0.8460990786552429, 0.8143693804740906, 0.9138779044151306, 0.9075807929039001, 0.9000207185745239, 0.9532613158226013, 0.9214703440666199, 0.9547213912010193, 0.9368969798088074, 0.9206411242485046, 0.9149274230003357, 0.9222517609596252, 0.8390894532203674, 0.8951126933097839, 0.8044940233230591, 0.8979001641273499, 0.8727086186408997, 0.869292676448822, 0.8827722668647766, 0.8631779551506042, 0.8986988067626953, 0.8237484693527222, 0.8694339394569397, 0.9581305980682373, 0.8593661785125732, 0.8116710782051086, 0.8567408919334412, 0.8325729370117188, 0.84066790342330

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_

flores+: 0.8893899539767679 Prediction([('scores', [0.8902938365936279, 0.9098991751670837, 0.9004725217819214, 0.8544191718101501, 0.9019644856452942, 0.9072950482368469, 0.9495924711227417, 0.9103862047195435, 0.9402479529380798, 0.921598494052887, 0.8602409362792969, 0.9293778538703918, 0.9688893556594849, 0.853983461856842, 0.8805211186408997, 0.9088164567947388, 0.9390103220939636, 0.9209024906158447, 0.9511423707008362, 0.8836107850074768, 0.7731045484542847, 0.7360331416130066, 0.9285632967948914, 0.883586585521698, 0.9035694003105164, 0.8397422432899475, 0.9048953652381897, 0.9308823347091675, 0.8723459243774414, 0.8989035487174988, 0.9113999605178833, 0.7100061774253845, 0.6966246962547302, 0.9093019366264343, 0.9077590107917786, 0.8997365236282349, 0.8790513277053833, 0.8742945790290833, 0.9161296486854553, 0.9293375611305237, 0.857886552810669, 0.9069087505340576, 0.8628986477851868, 0.9095414280891418, 0.8688846230506897, 0.9024907946586609, 0.9068729877471924, 0.8075796961

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_

from_training_dataset: 0.85144585447652 Prediction([('scores', [0.8536155223846436, 0.8212196826934814, 0.7289954423904419, 0.7871578335762024, 0.7241668701171875, 0.4683710038661957, 0.9015563130378723, 0.8976993560791016, 0.8330814242362976, 0.6907347440719604, 0.6662299633026123, 0.9012504816055298, 0.8239989876747131, 0.9580507874488831, 0.9288082718849182, 0.9307780265808105, 0.7687587141990662, 0.9281156659126282, 0.8984205722808838, 0.8494527339935303, 0.8920233845710754, 0.9629670977592468, 0.7897108793258667, 0.9254043698310852, 0.8925638794898987, 0.7847344279289246, 0.9281700849533081, 0.8143724799156189, 0.9453498721122742, 0.952032744884491, 0.8966856598854065, 0.8515797853469849, 0.5432989001274109, 0.7780823111534119, 0.8463820219039917, 0.9073676466941833, 0.9136314988136292, 0.8696392178535461, 0.922488272190094, 0.6240929961204529, 0.8988914489746094, 0.853981614112854, 0.9219463467597961, 0.8797938227653503, 0.8065828680992126, 0.864767849445343, 0.9204505085945129, 

# BERT SCORE

In [28]:
import evaluate
import numpy as np
def get_bert_score(predictions_list, reference_list):
    bertscore = evaluate.load("bertscore")
    results = bertscore.compute(predictions=predictions_list, references=reference_list, lang="ro")
    return results


def read_and_get_results_BERT(reference_folders, prediction_folders):
  results_dict["BERT"] = dict()
  for i in range(len(reference_folders)):
    reference_file_path = reference_folders[i] + "/" + reference_file_name
    prediction_file_path = prediction_folders[i] + "/" + prediction_file_name

    name_of_dataset = reference_folders[i].split("/")[-1]

    lines_reference = None
    lines_prediction = None

    with open(reference_file_path, "r") as file:
        lines_reference = [line.strip() for line in file]

    with open(prediction_file_path, "r") as file:
        lines_prediction = [line.strip() for line in file if line.strip()]
        #lines_prediction = [
        #    line.strip() if line.strip() else " "  # Replace empty lines with a space
        #    for line in file
        #]

    score = get_bert_score(lines_prediction, lines_reference)

    print(  name_of_dataset + ": " + str(np.mean(score['precision'])))
    results_dict["BERT"][name_of_dataset] = np.mean(score['precision'])

In [29]:
read_and_get_results_BERT(reference_folders, prediction_folders)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

arabia_saudita_guvern: 0.8282724581067524
conventia_geneva: 0.8400808697397059
drepturile_copilului: 0.8067204155680221
drepturile_economice_culturale: 0.8775740265846252
drepturile_omului: 0.9290794903718973
ksaembassyrou: 0.8880752401296482
statutul_refugiatilor: 0.7732023923410534
flores+: 0.8793413786137235
from_training_dataset: 0.8352751142638071


In [30]:
dataset_names = {
    'arabia_saudita_guvern': 'Saudi-Romania Econ',
    'conventia_geneva': 'Geneva Convention',
    'drepturile_copilului': 'Children Rights',
    'drepturile_economice_culturale': 'Economic Rights',
    'drepturile_omului': 'Human Rights',
    'ksaembassyrou': 'Saudi Embassy',
    'statutul_refugiatilor': 'Refugee Status',
    'flores+': 'FLORES+',
    'from_training_dataset': 'In-Domain dataset',
}

table_header = r"""\begin{table*}[h!]
\centering
\begin{tabular}{|l|r|r|r|r|}
\hline
\textbf{Dataset} & \textbf{BLEU Score} & \textbf{CHRF++ Score} & \textbf{WMT22-comet-da} & \textbf{BertScore} \\ \hline
"""

table_rows = []

bleu_scores_all = []
chrf_scores_all = []
comet_scores_all = []
bert_scores_all = []

for key, dataset in dataset_names.items():
    bleu_score = results_dict['BLEU'][key]
    chrf_score = results_dict['CHRF'][key]
    comet_score = results_dict['COMET'][key]
    bert_score = results_dict['BERT'][key]
    table_rows.append(f"{dataset} & {bleu_score:.2f} & {chrf_score:.2f} & {comet_score:.4f} & {bert_score:.4f} \\\\ \\hline")

    if key not in ["FLORES+", "In-Domain dataset"]:
        bleu_scores_all.append(bleu_score)
        chrf_scores_all.append(chrf_score)
        comet_scores_all.append(comet_score)
        bert_scores_all.append(bert_score)

table_footer = r"""\end{tabular}
\caption{TABLE_NAME}
\label{tab:CAPTION}
\end{table*}
"""

latex_table = table_header + "\n".join(table_rows) + "\n" + table_footer

print(latex_table)
print(f"Average BLEU Score: {np.mean(bleu_scores_all):.2f}")
print(f"Average CHRF++ Score: {np.mean(chrf_scores_all):.2f}")
print(f"Average WMT22-comet-da Score: {np.mean(comet_scores_all):.4f}")
print(f"Average BertScore Score: {np.mean(bert_scores_all):.4f}")
print(f"Model: {MODEL_NAME}")


\begin{table*}[h!]
\centering
\begin{tabular}{|l|r|r|r|r|}
\hline
\textbf{Dataset} & \textbf{BLEU Score} & \textbf{CHRF++ Score} & \textbf{WMT22-comet-da} & \textbf{BertScore} \\ \hline
Saudi-Romania Econ & 30.86 & 62.91 & 0.8445 & 0.8283 \\ \hline
Geneva Convention & 21.94 & 57.44 & 0.8603 & 0.8401 \\ \hline
Children Rights & 22.77 & 57.95 & 0.8842 & 0.8067 \\ \hline
Economic Rights & 20.06 & 59.95 & 0.9165 & 0.8776 \\ \hline
Human Rights & 28.11 & 61.93 & 0.9470 & 0.9291 \\ \hline
Saudi Embassy & 39.18 & 70.77 & 0.8811 & 0.8881 \\ \hline
Refugee Status & 14.36 & 50.09 & 0.8459 & 0.7732 \\ \hline
FLORES+ & 34.16 & 60.09 & 0.8894 & 0.8793 \\ \hline
In-Domain dataset & 27.71 & 54.41 & 0.8514 & 0.8353 \\ \hline
\end{tabular}
\caption{TABLE_NAME}
\label{tab:CAPTION}
\end{table*}

Average BLEU Score: 26.57
Average CHRF++ Score: 59.50
Average WMT22-comet-da Score: 0.8800
Average BertScore Score: 0.8508
Model: GTranslate
